In [ ]:
import pandas as pd
import math

In [ ]:
### 30yrs_weather_data_agg.csv file was created with below query. 30 years weather data was aggregated at grid level, initially it is on daily level.
### avg_pcp and avg_acc_snow were first sum across each year and then aggregated at grid level.
'''
select cc.*, bb.avg_pcp, bb.avg_acc_snow from
(SELECT grid_key, latitude, longitude, avg(air_2m) avg_temp, max(air_2m) max_temp, max(apcp) max_pcp, max(weasd) max_acc_snow,
min(air_2m) min_temp, min(apcp) min_pcp, min(weasd) min_acc_snow,
stddev(air_2m) stddev_temp, stddev(apcp) stddev_pcp, stddev(weasd) stddev_acc_snow FROM narr_climate group by grid_key, latitude, longitude)cc

left outer join

(select grid_key, latitude, longitude, avg(sum_pcp) avg_pcp, sum(sum_acc_snow) avg_acc_snow from 
(SELECT grid_key, latitude, longitude, year(hist_date), sum(apcp) sum_pcp, sum(weasd) sum_acc_snow
FROM narr_climate group by grid_key, latitude, longitude, year(hist_date)) aa group by grid_key, latitude, longitude)bb

on cc.grid_key=bb.grid_key and
cc.latitude=bb.latitude and
cc.longitude=bb.longitude

'''

In [ ]:
grid_keys1=pd.read_csv("30yrs_weather_data_agg.csv")
stores=pd.read_csv("store_attributes.csv")

grid_keys=grid_keys1[['grid_key','latitude','longitude']]
stores=stores[['store_code','store_latitude_degree','store_longitude_degree']]

In [ ]:
def rad(coord):
    
    '''
    converts lat/lon coordinates to radians
    '''
    
    rad = math.radians(coord)
   
    return rad


def get_distance(lat1, lon1, lat2, lon2):
    
    '''
    calculates arbitrary distance measure between 2 lat/lon pairs used for nearest grid key approximation
    '''
    
    x = (rad(lon2) - rad(lon1)) * math.cos(rad(lat1))
    y = rad(lat2) - rad(lat1)
    d = (x**2 + y**2)**(1/2)
    
    return d


def find_closest_on_grid(store, grid_ref):
    
    '''
    finds the closest NARR grid point for a given store 
    '''
    
    dist = []
    storeid = store[0]
    lat = store[1]
    lon = store[2]
    
    for g in grid_ref:
        d = get_distance(lat, lon, g[1], g[2])
        dist.append(d)
    
    closest_ind = dist.index(min(dist))
    
    closest_grid_key = int(grid_ref[closest_ind][0])
    closest_lat = grid_ref[closest_ind][1]
    closest_lon = grid_ref[closest_ind][2]
    
    out = [closest_grid_key, closest_lat, closest_lon]
    
    return out

In [ ]:
# convert data frames to iterables
grid_keys = grid_keys.values.tolist()
stores = stores.values.tolist()

In [ ]:
# find closest weather grid point for each store
results = []

for i,store in enumerate(stores):
    
    if i % 100 == 0:
        print(i)
    
    closest = find_closest_on_grid(store, grid_keys)
    
    results.append([store[0], store[1], store[2], closest[0], closest[1], closest[2]])

In [ ]:
df = pd.DataFrame(results, columns=['store_code','store_latitude','store_longitude', \
                                    'grid_key','grid_latitude','grid_longitude'])
#df.head()

In [ ]:
df=pd.merge(df,grid_keys1,on=['grid_key'], how='left')
#df

In [ ]:
df.drop(['latitude', 'longitude'], axis=1, inplace=True)
#df

In [ ]:
df.to_csv("store_hist_weather_info.csv")